In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
bace_chemberta2_valid2 = pd.read_csv('./chemberta2/results/bace/chemberta2_valid2_bace_1_predictions.csv')
bace_molformer_valid2 = pd.read_csv('./molformer/results/bace/molformer_valid2_bace_1_epoch49.csv')
bace_molbert_valid2 = pd.read_csv('./molbert/results/bace/molbert_valid2_bace_1.csv')

# Load the test data for each model
bace_chemberta2_test = pd.read_csv('./chemberta2/results/bace/chemberta2_test_bace_1_predictions.csv')
bace_molformer_test = pd.read_csv('./molformer/results/bace/molformer_test_bace_1_epoch49.csv')
bace_molbert_test = pd.read_csv('./molbert/results/bace/molbert_test_bace_1.csv')

# features

# Load the features from chemberta
bace_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/bace/chemberta2_valid2_bace_1_features.csv')
bace_chemberta2_features_test = pd.read_csv('./chemberta2/features/bace/chemberta2_test_bace_1_features.csv')

# Load the features from molformer
bace_molformer_features_valid2 = pd.read_csv('./molformer/features/bace/molformer_valid2_bace_1_features.csv')
bace_molformer_features_test = pd.read_csv('./molformer/features/bace/molformer_test_bace_1_features.csv')

# Load the features from molbert
bace_molbert_features_valid2 = pd.read_csv('./molbert/features/bace/molbert_valid2_bace_1_features.csv')
bace_molbert_features_test = pd.read_csv('./molbert/features/bace/molbert_test_bace_1_features.csv')

For BACE (Classification)

In [2]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Preparing the actual and predicted values
# Chemberta2
bace_chemberta_actual = bace_chemberta2_test['Class']
bace_chemberta_pred = bace_chemberta2_test['y_pred']
bace_chemberta_probs = bace_chemberta2_test[['softmax_class_0_prob', 'softmax_class_1_prob']]

# Molformer
bace_molformer_actual = bace_molformer_test['Actual']
bace_molformer_pred = (bace_molformer_test['Prob_Class_1'] > 0.5).astype(int)
bace_molformer_probs = bace_molformer_test[['Prob_Class_0', 'Prob_Class_1']]

# Molbert
bace_molbert_actual = bace_molbert_test['target']
bace_molbert_pred = bace_molbert_test['pred']
bace_molbert_probs = bace_molbert_test['prob']

# Calculating metrics
bace_metrics_results = {}

for model_name, actual, pred, probs in [("Chemberta2", bace_chemberta_actual, bace_chemberta_pred, bace_chemberta_probs['softmax_class_1_prob']),
                                         ("Molformer", bace_molformer_actual, bace_molformer_pred, bace_molformer_probs['Prob_Class_1']),
                                         ("Molbert", bace_molbert_actual, bace_molbert_pred, bace_molbert_probs)]:
    bace_metrics_results[model_name] = {
        "Accuracy": accuracy_score(actual, pred),
        "F1 Score": f1_score(actual, pred),
        "ROC-AUC": roc_auc_score(actual, probs),
        "PR-AUC": average_precision_score(actual, probs)
    }

bace_metrics_results

{'Chemberta2': {'Accuracy': 0.6535947712418301,
  'F1 Score': 0.6442953020134228,
  'ROC-AUC': 0.7465949820788532,
  'PR-AUC': 0.8058944338111258},
 'Molformer': {'Accuracy': 0.6797385620915033,
  'F1 Score': 0.6797385620915032,
  'ROC-AUC': 0.8521505376344086,
  'PR-AUC': 0.8774798970672688},
 'Molbert': {'Accuracy': 0.7189542483660131,
  'F1 Score': 0.7361963190184049,
  'ROC-AUC': 0.775089605734767,
  'PR-AUC': 0.8548501462313669}}

In [3]:
# check shapes
print(bace_chemberta2_valid2.shape)
print(bace_molformer_valid2.shape)
print(bace_molbert_valid2.shape)
print(bace_chemberta2_features_valid2.shape)
print(bace_molformer_features_valid2.shape)
print(bace_molbert_features_valid2.shape)

(305, 8)
(305, 5)
(305, 4)
(305, 386)
(305, 769)
(305, 769)


In [4]:
# create a new dataframe with one column of bace_chemberta2_valid2['softmax_class_1_prob']
bace_chemberta2_prob = pd.DataFrame({'chemberta2': bace_chemberta2_valid2['softmax_class_1_prob']})
bace_chemberta2_prob.reset_index(drop=True, inplace=True)

# create a new dataframe with one column of bace_molformer_valid2['Prob_Class_1']
bace_molformer_prob = pd.DataFrame({'molformer': bace_molformer_valid2['Prob_Class_1']})
bace_molformer_prob.reset_index(drop=True, inplace=True)

# create a new dataframe with one column of bace_molbert_valid2['Probabilities']
bace_molbert_prob = pd.DataFrame({'molbert': bace_molbert_valid2['prob']})
bace_molbert_prob.reset_index(drop=True, inplace=True)

# concatenate the three dataframes
bace_prob = pd.concat([bace_chemberta2_prob, bace_molformer_prob, bace_molbert_prob], axis=1)

# do the same for features bace_chemberta2_features_valid2.iloc[:, 2:]
bace_chemberta2_features = pd.DataFrame(bace_chemberta2_features_valid2.iloc[:, 2:])
bace_chemberta2_features.reset_index(drop=True, inplace=True)
bace_molformer_features = pd.DataFrame(bace_molformer_features_valid2.iloc[:, 1:])
bace_molformer_features.reset_index(drop=True, inplace=True)
bace_molbert_features = pd.DataFrame(bace_molbert_features_valid2.iloc[:, 1:])
bace_molbert_features.reset_index(drop=True, inplace=True)

bace_features = pd.concat([bace_chemberta2_features, bace_molformer_features, bace_molbert_features], axis=1)

# combine the features and probabilities
bace_X_ensemble_valid2 = pd.concat([bace_prob, bace_features], axis=1)

bace_y_ensemble_valid2 = bace_chemberta2_valid2['Class']

In [5]:
# do the same for test probs and features
bace_chemberta2_prob = pd.DataFrame({'chemberta2': bace_chemberta2_test['softmax_class_1_prob']})
bace_chemberta2_prob.reset_index(drop=True, inplace=True)
bace_molformer_prob = pd.DataFrame({'molformer': bace_molformer_test['Prob_Class_1']})
bace_molformer_prob.reset_index(drop=True, inplace=True)
bace_molbert_prob = pd.DataFrame({'molbert': bace_molbert_test['prob']})
bace_molbert_prob.reset_index(drop=True, inplace=True)
bace_prob = pd.concat([bace_chemberta2_prob, bace_molformer_prob, bace_molbert_prob], axis=1)

bace_chemberta2_features = pd.DataFrame(bace_chemberta2_features_test.iloc[:, 2:])
bace_chemberta2_features.reset_index(drop=True, inplace=True)
bace_molformer_features = pd.DataFrame(bace_molformer_features_test.iloc[:, 1:])
bace_molformer_features.reset_index(drop=True, inplace=True)
bace_molbert_features = pd.DataFrame(bace_molbert_features_test.iloc[:, 1:])
bace_molbert_features.reset_index(drop=True, inplace=True)
bace_features = pd.concat([bace_chemberta2_features, bace_molformer_features, bace_molbert_features], axis=1)

bace_X_ensemble_test = pd.concat([bace_prob, bace_features], axis=1)

bace_y_ensemble_test = bace_chemberta2_test['Class']

In [6]:
# use standard scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
bace_X_ensemble_valid2_scaled = scaler.fit_transform(bace_X_ensemble_valid2)
bace_X_ensemble_test_scaled = scaler.transform(bace_X_ensemble_test)

# transform back to dataframe
bace_X_ensemble_valid2_scaled = pd.DataFrame(bace_X_ensemble_valid2_scaled, columns=bace_X_ensemble_valid2.columns)
bace_X_ensemble_test_scaled = pd.DataFrame(bace_X_ensemble_test_scaled, columns=bace_X_ensemble_test.columns)


In [7]:
import numpy as np
from group_lasso import LogisticGroupLasso
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Example data loading - Ensure your dataset is loaded and scaled
# bace_X_ensemble_valid2_scaled, bace_y_ensemble_valid2 = load_your_data()

# Define the groups for features
n_features = bace_X_ensemble_valid2_scaled.shape[1]
groups = np.zeros(n_features, dtype=int)
groups[:3] = 1  # First three features as one group
groups[3:] = 2  # Rest of the features as another group

# Define the range of group_reg values to try
group_reg_values = np.logspace(-3, 1, 5)  # Example range from 0.001 to 10

# Setup cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

# Setup the GridSearchCV with LogisticGroupLasso
model = LogisticGroupLasso(
    groups=groups,
    l1_reg=0,  # No L1 regularization
    scale_reg='none',  # No automatic scaling
    supress_warning=True,
    tol=1e-2,
    random_state=0,
)

param_grid = {'group_reg': group_reg_values}
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=cv, scoring='roc_auc')

# Fit GridSearchCV
grid_search.fit(bace_X_ensemble_valid2_scaled, bace_y_ensemble_valid2)

# Best model
best_model = grid_search.best_estimator_

# Predictions and probabilities on a test set
# Assuming bace_X_ensemble_test_scaled and bace_y_ensemble_test are available
bace_group_lasso_pred = best_model.predict(bace_X_ensemble_test_scaled)
bace_group_lasso_probs = best_model.predict_proba(bace_X_ensemble_test_scaled)[:, 1]

# Evaluate the best model
bace_two_groups_lasso_metrics = {
    "Accuracy": accuracy_score(bace_y_ensemble_test, bace_group_lasso_pred),
    "F1 Score": f1_score(bace_y_ensemble_test, bace_group_lasso_pred),
    "ROC-AUC": roc_auc_score(bace_y_ensemble_test, bace_group_lasso_probs),
    "PR-AUC": average_precision_score(bace_y_ensemble_test, bace_group_lasso_probs)
}

bace_two_groups_lasso_metrics

# # Print best model results
# print("Best Group Reg:", grid_search.best_params_['group_reg'])
# print("Metrics:", bace_two_groups_lasso_metrics)

# # Print the coefficients of the best model
# print("Coefficients:", best_model.coef_)


{'Accuracy': 0.7124183006535948,
 'F1 Score': 0.725,
 'ROC-AUC': 0.8261648745519713,
 'PR-AUC': 0.8541481619100135}

In [8]:
import numpy as np
from group_lasso import LogisticGroupLasso
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Assume data is already loaded into bace_X_ensemble_valid2_scaled and bace_y_ensemble_valid2
# Define the groups for each feature
n_features = bace_X_ensemble_valid2_scaled.shape[1]
groups = np.zeros(n_features, dtype=int)
groups[:3] = 1  # First three features as one group
groups[3:3+384] = 2  # Next 384 features as another group
groups[3+384:3+384+768] = 3  # Next 768 features as third group
groups[3+384+768:] = 4  # Remaining features as fourth group

# Setup cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
param_grid = {'group_reg': np.logspace(-3, 1, 5)}  # Define a range of group_reg values

# Initialize the LogisticGroupLasso model within GridSearchCV
grid_search = GridSearchCV(
    estimator=LogisticGroupLasso(
        groups=groups,
        l1_reg=0,
        scale_reg='none',
        supress_warning=True,
        tol=1e-2,
        random_state=0,
    ),
    param_grid=param_grid,
    cv=cv,
    scoring='roc_auc'
)

# Fit GridSearchCV
grid_search.fit(bace_X_ensemble_valid2_scaled, bace_y_ensemble_valid2)

# Best model and parameters
best_model = grid_search.best_estimator_
print("Best group_reg:", grid_search.best_params_['group_reg'])

# Predict and evaluate using the best model
bace_group_lasso_pred = best_model.predict(bace_X_ensemble_test_scaled)
bace_group_lasso_probs = best_model.predict_proba(bace_X_ensemble_test_scaled)[:, 1]

# Calculate metrics
bace_four_groups_lasso_metrics = {
    "Accuracy": accuracy_score(bace_y_ensemble_test, bace_group_lasso_pred),
    "F1 Score": f1_score(bace_y_ensemble_test, bace_group_lasso_pred),
    "ROC-AUC": roc_auc_score(bace_y_ensemble_test, bace_group_lasso_probs),
    "PR-AUC": average_precision_score(bace_y_ensemble_test, bace_group_lasso_probs)
}

bace_four_groups_lasso_metrics

# Print coefficients for interpretation
# coefficients = best_model.coef_
# for i in range(1, 5):  # Display coefficients grouped by their groups
#     group_coefs = coefficients[groups == i]
#     print(f"Group {i} Coefficients:")
#     print(group_coefs)

Best group_reg: 0.1


{'Accuracy': 0.7058823529411765,
 'F1 Score': 0.7096774193548387,
 'ROC-AUC': 0.8286738351254481,
 'PR-AUC': 0.8426376943530206}

In [9]:
bace_X_ensemble_valid2_selected = bace_X_ensemble_valid2_scaled
bace_X_ensemble_test_selected = bace_X_ensemble_test_scaled
# check shapes
print(bace_X_ensemble_valid2_selected.shape)
print(bace_X_ensemble_test_selected.shape)

(305, 1923)
(153, 1923)


In [10]:
# Initialize and train the SVM model
from sklearn.svm import SVC

bace_svm_model = SVC(probability=True)
bace_svm_model.fit(bace_X_ensemble_valid2_selected, bace_y_ensemble_valid2)

# Predict the test set
bace_svm_pred = bace_svm_model.predict(bace_X_ensemble_test_selected)
bace_svm_probs = bace_svm_model.predict_proba(bace_X_ensemble_test_selected)

# Calculate the metrics
bace_svm_metrics = {
    'Accuracy': accuracy_score(bace_y_ensemble_test, bace_svm_pred),
    'F1 Score': f1_score(bace_y_ensemble_test, bace_svm_pred),
    'ROC-AUC': roc_auc_score(bace_y_ensemble_test, bace_svm_probs[:, 1]),
    "PR-AUC": average_precision_score(bace_y_ensemble_test, bace_svm_probs[:, 1])
}

bace_svm_metrics

{'Accuracy': 0.6274509803921569,
 'F1 Score': 0.5839416058394161,
 'ROC-AUC': 0.860752688172043,
 'PR-AUC': 0.8866411394096438}

In [11]:
# initailize and use a 5-fold cross-validation to tune the hyperparameters of a random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

bace_rf_model = RandomForestClassifier(random_state=0)

bace_rf_model.fit(bace_X_ensemble_valid2_selected, bace_y_ensemble_valid2)

# Predict the test set
bace_rf_best_pred = bace_rf_model.predict(bace_X_ensemble_test_selected)
bace_rf_best_probs = bace_rf_model.predict_proba(bace_X_ensemble_test_selected)

# Calculate the metrics
bace_rf_best_metrics = {
    "Accuracy": accuracy_score(bace_y_ensemble_test, bace_rf_best_pred),
    "F1 Score": f1_score(bace_y_ensemble_test, bace_rf_best_pred),
    "ROC-AUC": roc_auc_score(bace_y_ensemble_test, bace_rf_best_probs[:, 1]),
    "PR-AUC": average_precision_score(bace_y_ensemble_test, bace_rf_best_probs[:, 1])
}

bace_rf_best_metrics

{'Accuracy': 0.6339869281045751,
 'F1 Score': 0.6,
 'ROC-AUC': 0.8420250896057349,
 'PR-AUC': 0.8729096193143699}

In [12]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Define the hyperparameter space using continuous distributions
bace_xgb_hyperopt_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 200, 50),
    'max_depth': hp.quniform('max_depth', 3, 7, 2),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

# Convert float outputs of hp.quniform to int for certain parameters
def objective(params):
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    
    model = xgb.XGBClassifier(**params, random_state=0)
    
    # Create a scorer that calculates the roc_auc score using predicted probabilities
    roc_auc = make_scorer(roc_auc_score, needs_proba=True)
    score = cross_val_score(model, bace_X_ensemble_valid2_selected, bace_y_ensemble_valid2, scoring=roc_auc, cv=5)
    
    # Minimize the negative ROC AUC score
    return {'loss': -score.mean(), 'status': STATUS_OK}

# Run the Bayesian optimization
trials = Trials()
bace_xgb_best_params = fmin(fn=objective, 
                            space=bace_xgb_hyperopt_space, 
                            algo=tpe.suggest, 
                            max_evals=50, 
                            trials=trials,
                            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bace_xgb_best_params)

 20%|██        | 10/50 [00:57<03:50,  5.76s/trial, best loss: -0.9388131868131868]
Best hyperparameters: {'colsample_bytree': 0.6253001900637998, 'learning_rate': 0.005847905873763658, 'max_depth': 4.0, 'n_estimators': 100.0, 'subsample': 0.8127114352855627}


In [13]:
# Convert parameters obtained from Hyperopt to the correct data type
bace_xgb_best_params['n_estimators'] = int(bace_xgb_best_params['n_estimators'])
bace_xgb_best_params['max_depth'] = int(bace_xgb_best_params['max_depth'])

# Initialize and train the XGBoost model with the best parameters
bace_xgb_model = xgb.XGBClassifier(**bace_xgb_best_params, random_state=0)
bace_xgb_model.fit(bace_X_ensemble_valid2_selected, bace_y_ensemble_valid2)

# Predict the test set
bace_xgb_best_pred = bace_xgb_model.predict(bace_X_ensemble_test_selected)
bace_xgb_best_probs = bace_xgb_model.predict_proba(bace_X_ensemble_test_selected)

# Calculate the metrics
bace_xgb_best_metrics = {
    "Accuracy": accuracy_score(bace_y_ensemble_test, bace_xgb_best_pred),
    "F1 Score": f1_score(bace_y_ensemble_test, bace_xgb_best_pred),
    "ROC-AUC": roc_auc_score(bace_y_ensemble_test, bace_xgb_best_probs[:, 1]),
    "PR-AUC": average_precision_score(bace_y_ensemble_test, bace_xgb_best_probs[:, 1])
}

bace_xgb_best_metrics

{'Accuracy': 0.7058823529411765,
 'F1 Score': 0.713375796178344,
 'ROC-AUC': 0.839426523297491,
 'PR-AUC': 0.8754713161742003}

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

torch.manual_seed(0)

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params):
    kf = KFold(n_splits=5)
    roc_aucs = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = nn.BCELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.values.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.values.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            roc_auc = roc_auc_score(y_val_tensor.numpy(), outputs.numpy())
            roc_aucs.append(roc_auc)

    avg_roc_auc = np.mean(roc_aucs)
    return {'loss': -avg_roc_auc, 'status': STATUS_OK}  # Maximize ROC AUC by minimizing the negative ROC AUC

# Hyperparameter space
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

X = bace_X_ensemble_valid2_selected
y = bace_y_ensemble_valid2

# Run Bayesian optimization
trials = Trials()
bace_nn_best_params = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=50,
            trials=trials,
            early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", bace_nn_best_params)


 38%|███▊      | 19/50 [01:30<02:27,  4.74s/trial, best loss: -0.9070756217466742]
Best hyperparameters: {'dropout_rate': 0.19633897351920887, 'learning_rate': 0.00041510185129241473, 'num_layers': 1.0, 'num_neurons': 217.0}


In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import roc_auc_score

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1), nn.Sigmoid()]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Convert parameters to the correct format if necessary
bace_nn_best_params = {
    'num_layers': int(bace_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons': int(bace_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': bace_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': bace_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(bace_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(bace_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(bace_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(bace_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=bace_X_ensemble_valid2_selected.shape[1], num_layers=bace_nn_best_params['num_layers'], 
                 num_neurons=bace_nn_best_params['num_neurons'], dropout_rate=bace_nn_best_params['dropout_rate'])

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=bace_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = (outputs > 0.5).float()

    # Calculate metrics
    accuracy = accuracy_score(y_test_tensor.numpy(), predictions.numpy())
    f1 = f1_score(y_test_tensor.numpy(), predictions.numpy())
    roc_auc = roc_auc_score(y_test_tensor.numpy(), outputs.numpy())
    pr_auc = average_precision_score(y_test_tensor.numpy(), outputs.numpy())

    bace_nn_metrics = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'ROC-AUC': roc_auc,
        'PR-AUC': pr_auc
    }

bace_nn_metrics

{'Accuracy': 0.6797385620915033,
 'F1 Score': 0.6754966887417218,
 'ROC-AUC': 0.8525089605734767,
 'PR-AUC': 0.8828610080606898}

In [16]:
# report all the metrics for ct
bace_metrics_results["Two Groups Lasso"] = bace_two_groups_lasso_metrics
bace_metrics_results["Four Groups Lasso"] = bace_four_groups_lasso_metrics
bace_metrics_results["SVM"] = bace_svm_metrics
bace_metrics_results["Random Forest"] = bace_rf_best_metrics
bace_metrics_results["XGBoost"] = bace_xgb_best_metrics
bace_metrics_results["Neural Network"] = bace_nn_metrics

bace_metrics_df = pd.DataFrame(bace_metrics_results).T

# keep 3 digits after the decimal point
bace_metrics_df = bace_metrics_df.round(3)

# export as csv
bace_metrics_df.to_csv('./split1_bace_metrics_grouplasso.csv')